In this lab, you will continue exploring Model subclassing by building a more complex architecture.

Residual Networks make use of skip connections to make deep models easier to train.
* There are branches as well as many repeating blocks of layers in this type of network.
* You can define a model class to help organize this more complex code, and to make it easier to re-use your code when building the model.
* As before, you will inherit from the Model class so that you can make use of the other built-in methods that Keras provides.


In [5]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds

Implement Model subclasses

As shown in the lectures, you will first implement the Identity Block which contains the skip connections (i.e. the add() operation below. This will also inherit the Model class and implement the __init__() and call() methods.

In [29]:
class IdentityBlock(Model):

  def __init__(self, filters, kernel_size):
    super(IdentityBlock, self).__init__()
    self.conv1 = layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same')
    self.bn1 = layers.BatchNormalization()
    self.conv2 = layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same')
    self.bn2 = layers.BatchNormalization()
    self.activation = layers.Activation('relu')
    self.add = layers.Add()

  def call(self, input):
    x = self.conv1(input)
    x = self.bn1(x)
    x = self.activation(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.activation(x)
    x = self.add([input, x])
    x = self.activation(x)
    return x



From there, you can build the rest of the ResNet model.
* You will call your IdentityBlock class two times below and that takes care of inserting those blocks of layers into this network.



In [30]:
class ResNet(Model):

  def __init__(self, num_classes):
    super(ResNet, self).__init__()
    self.conv = layers.Conv2D(filters=64, kernel_size=(7, 7))
    self.bn = layers.BatchNormalization()
    self.max_pool = layers.MaxPool2D(pool_size=(3, 3))
    self.id1 = IdentityBlock(64, 3)
    self.id2 = IdentityBlock(64, 3)
    self.global_pool = layers.GlobalAveragePooling2D()
    self.classifer = layers.Dense(units=num_classes, activation='softmax')
    self.activation = layers.Activation('relu')

  def call(self, input):
    x = self.conv(input)
    x = self.bn(x)
    x = self.activation(x)
    x = self.max_pool(x)
    x = self.id1(x)
    x = self.id2(x)
    x = self.global_pool(x)
    x = self.classifer(x)
    return x


## Training the Model

As mentioned before, inheriting the Model class allows you to make use of the other APIs that Keras provides, such as:
* training
* serialization
* evaluation

You can instantiate a Resnet object and train it as usual like below:

Note: If you have issues with training in the Coursera lab environment, you can also run this in Colab using the "open in colab" badge link.


In [31]:
# utility function to normalize the images and return (image, label) pairs.
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

# load and preprocess the dataset
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)

# create a ResNet instance with 10 output units for MNIST
resnet = ResNet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# train the model.
resnet.fit(dataset, epochs=1)

1875/1875 [==============================] - 145s 77ms/step - loss: 0.2888 - accuracy: 0.9292
